In [157]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from envs.humanoid import HumanoidTracking
import numpy as np
import mediapy as media
import jax.numpy as jp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Humanoid env

In [158]:
params = {
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 6,
    "clip_path": "clips/test_traj.p",
}

In [159]:
env = HumanoidTracking(params)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:344: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  link = jax.tree_map(lambda x: x[1:].copy(), link)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:374: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  motion = jax.tree_map(lambda *x: np.concatenate(x), *motions)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:378: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  limit = jax.tree_map(lambda *x: np.concatenate(x), *limits)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitatio

In [150]:
state = env.reset(jax.random.PRNGKey(0))

IndexError: f argument "3" with type "q" has length "63" which does not match the in_types[3] expected length of "28".

In [10]:
key = random.PRNGKey(0)
jit_step = jax.jit(env.step)
jit_reset = jax.jit(env.reset)

In [11]:
next = jit_reset(key)
print("starting rollout")
mu = 0
sigma = .1
rollout = []
for _ in range(500):
    _, key = jax.random.split(key)
    next = jit_step(next, mu + sigma * random.normal(key, shape=(env.sys.nu,)))
    rollout.append(next)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco

starting rollout


/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco

In [12]:
import mediapy as media
import os

os.environ["MUJOCO_GL"] = "glfw"
rollout_data = [s.pipeline_state for s in rollout]

video = env.render(rollout_data, camera='side', height=500, width=500)

media.show_video(video, fps=1.0 / env.dt)

This browser does not support the video tag.


In [20]:
state.obs.shape

(685,)

# COMIC Validation Observation

In [104]:
from dm_control import composer
from dm_control.locomotion import arenas
from dm_control.locomotion import walkers
from walker import Rat
from dm_control.locomotion.mocap import props
from dm_control.locomotion.tasks.reference_pose import tracking
from dm_control.locomotion.tasks.reference_pose import types
from dm_control.utils import io as resources
import os
import h5py
import pickle

In [129]:
# Define the path to your motion capture data file
file_name = 'clips/test_traj.h5' #'clips/all_snips_250.h5'
current_directory = os.getcwd()
TEST_FILE_PATH = os.path.join(current_directory, file_name)

with h5py.File(TEST_FILE_PATH, 'r') as f:
    dataset_keys = tuple(f.keys())
    dataset = types.ClipCollection(ids=dataset_keys,)

# Set up the mocap tracking task
task = tracking.MultiClipMocapTracking(
    walker=walkers.CMUHumanoidPositionControlled,
    arena=arenas.Floor(),
    ref_path=resources.GetResourceFilename(TEST_FILE_PATH),
    ref_steps=(1, 2, 3, 4, 5),
    min_steps=1,
    dataset=dataset,
    reward_type='comic',
)

# Initialize the environment
env = composer.Environment(task=task)
env.reset()

TimeStep(step_type=<StepType.FIRST: 0>, reward=None, discount=None, observation=OrderedDict([('reference_props_pos_global', array([], shape=(1, 0), dtype=float64)), ('reference_props_quat_global', array([], shape=(1, 0), dtype=float64)), ('walker/actuator_activation', array([], shape=(1, 0), dtype=float64)), ('walker/appendages_pos', array([[-4.39773112e-01,  3.19360239e-01,  2.15425874e-01,
         4.44568141e-01,  3.20134413e-01,  2.12678368e-01,
        -3.45976051e-01, -5.29474255e-01,  1.29103623e-01,
         3.51150601e-01, -5.32441339e-01,  1.28891607e-01,
        -2.27434610e-04,  4.03886326e-01,  2.49860801e-01]])), ('walker/body_height', array([0.10500313])), ('walker/end_effectors_pos', array([[-0.43977311,  0.31936024,  0.21542587,  0.44456814,  0.32013441,
         0.21267837, -0.34597605, -0.52947426,  0.12910362,  0.3511506 ,
        -0.53244134,  0.12889161]])), ('walker/joints_pos', array([[ 2.10732833e-01, -2.39701695e-04,  1.38906628e-05,
        -4.47694166e-02, -

In [47]:
env.observation_spec().keys()

odict_keys(['reference_props_pos_global', 'reference_props_quat_global', 'walker/actuator_activation', 'walker/appendages_pos', 'walker/body_height', 'walker/end_effectors_pos', 'walker/joints_pos', 'walker/joints_vel', 'walker/sensors_accelerometer', 'walker/sensors_force', 'walker/sensors_gyro', 'walker/sensors_torque', 'walker/sensors_touch', 'walker/sensors_velocimeter', 'walker/world_zaxis', 'walker/reference_rel_joints', 'walker/reference_rel_bodies_pos_global', 'walker/reference_rel_bodies_quats', 'walker/reference_rel_bodies_pos_local', 'walker/reference_ego_bodies_quats', 'walker/reference_rel_root_quat', 'walker/reference_rel_root_pos_local', 'walker/reference_appendages_pos', 'walker/clip_id', 'walker/velocimeter_control', 'walker/gyro_control', 'walker/joints_vel_control', 'walker/time_in_clip'])

In [48]:
env.observation_spec()['walker/end_effectors_pos']

Array(shape=(1, 12), dtype=dtype('float64'), name='walker/end_effectors_pos')

In [53]:
# Prints the h5 file hierarchy and the dataset shapes
def print_hierarchy(h5file, indent=""):
    for key, item in h5file.items():
        if isinstance(item, h5py.Group):
            print(f"{indent}{key} (Group)")
            print_hierarchy(item, indent + "  ")
        elif isinstance(item, h5py.Dataset):
            print(f"{indent}{key} (Dataset): {item.shape}")
            
filename = "clips/test_trajectories.h5"
with h5py.File(filename, "r") as f:
    print(f"Contents of {filename}:")
    print_hierarchy(f)

Contents of clips/test_trajectories.h5:
cmuv2019_001 (Group)
  props (Group)
    prop_0 (Group)
      angular_velocity (Dataset): (3, 10)
      position (Dataset): (3, 10)
      quaternion (Dataset): (4, 10)
      velocity (Dataset): (3, 10)
  walkers (Group)
    walker_0 (Group)
      angular_velocity (Dataset): (3, 10)
      appendages (Dataset): (15, 10)
      body_positions (Dataset): (90, 10)
      body_quaternions (Dataset): (120, 10)
      center_of_mass (Dataset): (3, 10)
      end_effectors (Dataset): (12, 10)
      joints (Dataset): (56, 10)
      joints_velocity (Dataset): (56, 10)
      markers (Group)
        marker_0 (Group)
        marker_1 (Group)
        marker_2 (Group)
        marker_3 (Group)
        marker_4 (Group)
      position (Dataset): (3, 10)
      quaternion (Dataset): (4, 10)
      scaling (Group)
      velocity (Dataset): (3, 10)
cmuv2019_002 (Group)
  props (Group)
    prop_0 (Group)
      angular_velocity (Dataset): (3, 10)
      position (Dataset): (3,

# File `.p` Conversion from COMIC `.h5`

In [59]:
def extract_h5_to_dict(file):
    """Recursively extract all datasets from the HDF5 group into a dictionary."""
    data_dict = {}
    for key in file.keys():
        if isinstance(file[key], h5py.Dataset):
            # Store data as a numpy array
            data_dict[key] = file[key][:]
        elif isinstance(file[key], h5py.Group):
            # Recursively process each group
            data_dict.update({f"{key}/{subkey}": subvalue for subkey, subvalue in extract_h5_to_dict(file[key]).items()})
    return data_dict

# Path to your .h5 file
file_path = 'clips/test_traj.h5'

# Open the .h5 file and extract all data into a dictionary
with h5py.File(file_path, 'r') as file:
    all_data_dict = extract_h5_to_dict(file)

In [69]:
all_data_dict.keys()

dict_keys(['cmuv2019_001/props/prop_0/angular_velocity', 'cmuv2019_001/props/prop_0/position', 'cmuv2019_001/props/prop_0/quaternion', 'cmuv2019_001/props/prop_0/velocity', 'cmuv2019_001/walkers/walker_0/angular_velocity', 'cmuv2019_001/walkers/walker_0/appendages', 'cmuv2019_001/walkers/walker_0/body_positions', 'cmuv2019_001/walkers/walker_0/body_quaternions', 'cmuv2019_001/walkers/walker_0/center_of_mass', 'cmuv2019_001/walkers/walker_0/end_effectors', 'cmuv2019_001/walkers/walker_0/joints', 'cmuv2019_001/walkers/walker_0/joints_velocity', 'cmuv2019_001/walkers/walker_0/position', 'cmuv2019_001/walkers/walker_0/quaternion', 'cmuv2019_001/walkers/walker_0/velocity', 'cmuv2019_002/props/prop_0/angular_velocity', 'cmuv2019_002/props/prop_0/position', 'cmuv2019_002/props/prop_0/quaternion', 'cmuv2019_002/props/prop_0/velocity', 'cmuv2019_002/walkers/walker_0/angular_velocity', 'cmuv2019_002/walkers/walker_0/appendages', 'cmuv2019_002/walkers/walker_0/body_positions', 'cmuv2019_002/walke

In [121]:
# Dictionary to store specific data following the provided structure
structured_data = {
    'angular_velocity': all_data_dict.get('cmuv2019_001/walkers/walker_0/angular_velocity', None).T,
    'appendages': all_data_dict.get('cmuv2019_001/walkers/walker_0/appendages', None).T,
    'body_positions': all_data_dict.get('cmuv2019_001/walkers/walker_0/body_positions', None).T,
    'body_quaternions': all_data_dict.get('cmuv2019_001/walkers/walker_0/body_quaternions', None).T,
    'center_of_mass': all_data_dict.get('cmuv2019_001/walkers/walker_0/center_of_mass', None).T,
    'end_effectors': all_data_dict.get('cmuv2019_001/walkers/walker_0/end_effectors', None).T,
    'joints': all_data_dict.get('cmuv2019_001/walkers/walker_0/joints', None).T,
    'joints_velocity': all_data_dict.get('cmuv2019_001/walkers/walker_0/joints_velocity', None).T,
    'markers': all_data_dict.get('cmuv2019_001/walkers/walker_0/markers', None),
    'position': all_data_dict.get('cmuv2019_001/walkers/walker_0/position', None).T,
    'quaternion': all_data_dict.get('cmuv2019_001/walkers/walker_0/quaternion', None).T,
    'scaling': all_data_dict.get('cmuv2019_001/walkers/walker_0/scaling', None),
    'velocity': all_data_dict.get('cmuv2019_001/walkers/walker_0/velocity', None).T
}

# Optionally, print the data for verification
for key, value in structured_data.items():
    print(f"{key}: {type(value)}")

angular_velocity: <class 'numpy.ndarray'>
appendages: <class 'numpy.ndarray'>
body_positions: <class 'numpy.ndarray'>
body_quaternions: <class 'numpy.ndarray'>
center_of_mass: <class 'numpy.ndarray'>
end_effectors: <class 'numpy.ndarray'>
joints: <class 'numpy.ndarray'>
joints_velocity: <class 'numpy.ndarray'>
markers: <class 'NoneType'>
position: <class 'numpy.ndarray'>
quaternion: <class 'numpy.ndarray'>
scaling: <class 'NoneType'>
velocity: <class 'numpy.ndarray'>


In [122]:
[np.array(structured_data[item]).shape for item in structured_data]

[(10, 3),
 (10, 15),
 (10, 90),
 (10, 120),
 (10, 3),
 (10, 12),
 (10, 56),
 (10, 56),
 (),
 (10, 3),
 (10, 4),
 (),
 (10, 3)]

In [123]:
[item for item in structured_data]

['angular_velocity',
 'appendages',
 'body_positions',
 'body_quaternions',
 'center_of_mass',
 'end_effectors',
 'joints',
 'joints_velocity',
 'markers',
 'position',
 'quaternion',
 'scaling',
 'velocity']

In [141]:
import mocap_preprocess as mp
mp.save_dataclass_pickle('clips/test_traj.p', structured_data)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/mocap_preprocess.py:340: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  data = jax.tree_map(lambda x: jp.array(x), data)


# Checking if shape mataches
1. for qpos stacks, `angular_velocity`, `veclocity` matches, `joints_velocity` doesn't

In [140]:
file_path = 'clips/humanoid_traj.p'

with open(file_path, 'rb') as file:
    data = pickle.load(file)

print(type(data))
print(data.joints_velocity.shape)

<class 'mocap_preprocess.ReferenceClip'>
(415, 21)


In [139]:
file_path = 'clips/test_traj.p'

with open(file_path, 'rb') as file:
    data = pickle.load(file)

print(type(data))
print(data.joints_velocity.shape)

<class 'mocap_preprocess.ReferenceClip'>
(10, 56)


# Our `humanoid_traj.p` to `.h5` For COMIC

In [107]:
cat = ['angular_velocity',
 'appendages',
 'body_positions',
 'body_quaternions',
 'center_of_mass',
 'end_effectors',
 'joints',
 'joints_velocity',
 'markers',
 'position',
 'quaternion',
 'scaling',
 'velocity']

# Path to the pickle file and the desired output HDF5 file path
pickle_file_path = 'clips/humanoid_traj.p'
hdf5_file_path = 'humanoid_traj.h5'

# Load the data from the pickle file
with open(pickle_file_path, 'rb') as file:
    data = pickle.load(file)

# Create an HDF5 file and store the data
with h5py.File(hdf5_file_path, 'w') as hdf_file:
    for mocap in cat:
        # Assuming each value is an array-like object that can be directly stored in HDF5
        if isinstance(value, np.ndarray):
            hdf_file.create_dataset(cat, data=data.mocap)
        else:
            print(f"Skipping key {key} as it is not an array.")
print(f"Data from {pickle_file_path} has been successfully written to {hdf5_file_path}.")

Skipping key velocity as it is not an array.
Skipping key velocity as it is not an array.
Skipping key velocity as it is not an array.
Skipping key velocity as it is not an array.
Skipping key velocity as it is not an array.
Skipping key velocity as it is not an array.
Skipping key velocity as it is not an array.
Skipping key velocity as it is not an array.
Skipping key velocity as it is not an array.
Skipping key velocity as it is not an array.
Skipping key velocity as it is not an array.
Skipping key velocity as it is not an array.
Skipping key velocity as it is not an array.
Data from clips/humanoid_traj.p has been successfully written to humanoid_traj.h5.
